In [1]:
import os
import re
import numpy as np
from skimage.transform import resize, rescale
from scipy import ndimage, misc
from matplotlib import pyplot
import matplotlib.pyplot as plt
np.random.seed(0)

In [2]:
from tensorflow.keras.layers import Conv2D, Dense, Input, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
print(tf.__version__)

2.1.0


In [4]:
# Enc schema mapping
input_data = Input(shape=(256, 256, 3))
enc_l1 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_data)
enc_l2 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(enc_l1)
enc_l3 = MaxPooling2D(padding='same')(enc_l2)
enc_l4 = Conv2D(96, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(enc_l3)

In [5]:
# Alternatively, MaxPooling2D can be redef for verbosity as :
# def max_pool_2x2(x):
#   return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
# Similarly, conv2D call can be redef for verbosity as :
# def conv2D(x, W):
#   return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')